## Web Scraping

In [1]:
import requests
import re
from bs4 import BeautifulSoup

import json

import time

1978-1979 season to the 2017-2018 season

# Generate URLs for all the seasons of interest from Basketball reference 

In [7]:
url = 'https://www.basketball-reference.com/leagues/NBA_'

In [10]:
def url_gen(url):
    url_list = []
    for i in range(1979, 2019):
        url_list.append(url + str(i) + '.html')
    return url_list

In [15]:
season_urls = url_gen(url)

In [16]:
season_2017 = season_urls[-1]

In [17]:
season_2017

'https://www.basketball-reference.com/leagues/NBA_2018.html'